In [ ]:
#terminal
pip install fuzzywuzzy
pip install python-Levenshtein


In [11]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
import re

# =============================================================================
# 1. O*NET SKILL EXTRACTION
# =============================================================================

def extract_onet_jobs_and_skills(skills_file_path="Skills.csv"):
    """Extract jobs and key skills from O*NET."""
    print("🔍 Extracting O*NET jobs and skills...")
    try:
        df = pd.read_csv(skills_file_path, sep=';', low_memory=False, encoding='utf-8')
        df.columns = df.columns.str.strip()

        if 'Data Value' in df.columns:
            df['Data Value'] = df['Data Value'].astype(str).str.replace(',', '.')
            df['Data Value'] = pd.to_numeric(df['Data Value'], errors='coerce')

        df = df[df['Scale Name'].str.lower() == 'importance']
        if df.empty:
            print("❌ No importance scale found")
            return {}

        job_skills = {}
        for title in df['Title'].unique():
            tdf = df[df['Title'] == title]
            important = [r['Element Name'] for _, r in tdf.iterrows() if pd.notna(r['Data Value']) and r['Data Value'] >= 3.0]
            job_skills[title.lower()] = " + ".join(important[:5]) if important else "General Skills"

        print(f"✅ Extracted {len(job_skills)} jobs with skills")
        return job_skills

    except Exception as e:
        print(f"❌ Error: {e}")
        return {}

# =============================================================================
# 2. JOB MATCHING (CUSTOM + FUZZY + FALLBACK)
# =============================================================================

def intelligent_job_matching(main_jobs, onet_jobs_skills):
    print("🔍 Matching jobs with O*NET...")
    results, onet_jobs = {}, list(onet_jobs_skills.keys())

    custom = {
        'investment analyst': 'financial analysts',
        'financial planner': 'personal financial advisors',
        'journalist': 'reporters and correspondents',
        'legal secretary': 'legal secretaries and administrative assistants',
        'aeronautical engineer': 'aerospace engineers',
        'software developer': 'software developers',
        'web developer': 'web developers',
        'data scientist': 'data scientists',
        'data analyst': 'operations research analysts',
        'teacher': 'secondary school teachers'
    }

    for job in main_jobs:
        jl = job.lower().strip()
        match = None
        if jl in custom and custom[jl].lower() in onet_jobs_skills:
            match = onet_jobs_skills[custom[jl].lower()]
        elif jl in onet_jobs:
            match = onet_jobs_skills[jl]
        else:
            fuzzy = process.extract(jl, onet_jobs, limit=3, scorer=fuzz.token_sort_ratio)
            for cand, score in fuzzy:
                if score >= 75:
                    match = onet_jobs_skills[cand]
                    break
        if not match:
            jkw = set(re.findall(r'\w+', jl))
            for oj in onet_jobs:
                okw = set(re.findall(r'\w+', oj))
                if len(jkw & okw) >= 2:
                    match = onet_jobs_skills[oj]
                    break
        results[job] = match

    matched = sum(1 for v in results.values() if v)
    print(f"✅ Matched {matched}/{len(main_jobs)} jobs ({matched/len(main_jobs)*100:.1f}%)")
    return results

# =============================================================================
# 3. MAIN PIPELINE
# =============================================================================

def run_skills_pipeline(main_data_path="prepared_data.csv", skills_path="Skills.csv"):
    print("🚀 RUNNING O*NET SKILLS PIPELINE")

    # Load data
    df = pd.read_csv(main_data_path)
    print(f"📥 Main dataset: {df.shape} ({df['Job Title'].nunique()} unique titles)")

    # Extract O*NET skills
    onet = extract_onet_jobs_and_skills(skills_path)
    if not onet:
        print("❌ Stopping: no O*NET data.")
        return None

    # Match
    matches = intelligent_job_matching(df['Job Title'].unique(), onet)

    # Prepare output
    keep = [c for c in ['Job Title','Industry','AI Impact Level','Median Salary (USD)',
                        'Automation Risk (%)','Growth (%)','Remote Work Ratio (%)'] if c in df.columns]
    final = df[keep].copy()
    final['Required_Skills'] = final['Job Title'].map(matches).fillna("No O*NET skills found")
    final['Skills_Count'] = final['Required_Skills'].apply(
        lambda x: len(x.split(' + ')) if x not in ["No O*NET skills found","General Skills"] else 0
    )

    # Clustering data
    cluster_cols = [c for c in ['Skills_Count','Automation Risk (%)','Growth (%)',
                                'Median Salary (USD)','Remote Work Ratio (%)'] if c in final.columns]
    cluster_df = final[cluster_cols].fillna(0)

    # Save
    final.to_csv("readyformodeling_with_skills.csv", index=False)
    cluster_df.to_csv("clustering_ready.csv", index=False)

    print(f"💾 Saved readyformodeling_with_skills.csv ({final.shape})")
    print(f"💾 Saved clustering_ready.csv ({cluster_df.shape})")
    print(f"✅ All {len(final)} jobs preserved")

    print("\n📊 Sample matches:")
    print(final[['Job Title','Required_Skills','Skills_Count']].head(10).to_string(index=False))
    return final, cluster_df


if __name__ == "__main__":
    df_final, cluster_data = run_skills_pipeline(
        main_data_path="prepared_data.csv",
        skills_path="data/Skills.csv"
    )
    if df_final is not None:
        print(f"Final dataset: {df_final.shape}, clustering: {cluster_data.shape}")
    else:
        print("\n failed")


🚀 RUNNING O*NET SKILLS PIPELINE
📥 Main dataset: (30000, 27) (639 unique titles)
🔍 Extracting O*NET jobs and skills...
✅ Extracted 894 jobs with skills
🔍 Matching jobs with O*NET...
✅ Matched 216/639 jobs (33.8%)
💾 Saved readyformodeling_with_skills.csv ((30000, 6))
💾 Saved clustering_ready.csv ((30000, 4))
✅ All 30000 jobs preserved

📊 Sample matches:
                            Job Title                                                                   Required_Skills  Skills_Count
                   investment analyst       Reading Comprehension + Active Listening + Writing + Speaking + Mathematics             5
                journalist, newspaper                                                             No O*NET skills found             0
                    financial planner       Reading Comprehension + Active Listening + Writing + Speaking + Mathematics             5
                      legal secretary Reading Comprehension + Active Listening + Writing + Speaking + Critical

In [12]:
df_final.head()


,Job Title,Median Salary (USD),Automation Risk (%),Remote Work Ratio (%),Required_Skills,Skills_Count
0,investment analyst,-1.395182,-0.760726,0.211404,Reading Comprehension + Active Listening + Wri...,5
1,"journalist, newspaper",1.225715,1.375642,-1.140171,No O*NET skills found,0
2,financial planner,1.544813,0.793470,1.449398,Reading Comprehension + Active Listening + Wri...,5
3,legal secretary,0.216677,1.731413,-1.655254,Reading Comprehension + Active Listening + Wri...,5
4,aeronautical engineer,-0.847490,-0.434863,0.135453,Reading Comprehension + Active Listening + Wri...,5
